<a href="https://colab.research.google.com/github/inkwar2019/Google-Colab-Notebook/blob/main/CNN_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
from keras.datasets import mnist
(a, b), (c, d) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = (list((a/255.0).reshape((-1,28,28,1))), list(b)), (list((c/255.0).reshape((-1,28,28,1))), list(d))

11501568/11490434 [==============================] - 0s 0us/step


# JSON for weight initialization

In [2]:
import json
import random

#conv01
with open("conv1.json", "w") as write_file:
  biases = []
  weights = []
  for i in range(32):
    biases.append(random.randint(0,100)/100)
    temp1 = []
    for j in range(1):
      temp2 = []
      for k in range(3):
        temp3 = []
        for l in range(3):
          temp3.append(random.randint(-500,500)/500)
        temp2.append(temp3)
      temp1.append(temp2)
    weights.append(temp1)
  data = {
      "biases":biases,
      "weights":weights
  }
  json.dump(data, write_file)

#conv02
with open("conv2.json", "w") as write_file:
  biases = []
  weights = []
  for i in range(64):
    biases.append(random.randint(0,100)/100)
    temp1 = []
    for j in range(32):
      temp2 = []
      for k in range(3):
        temp3 = []
        for l in range(3):
          temp3.append(random.randint(-500,500)/500)
        temp2.append(temp3)
      temp1.append(temp2)
    weights.append(temp1)
  data = {
      "biases":biases,
      "weights":weights
  }
  json.dump(data, write_file)

#Dense
with open("dense.json", "w") as write_file:
  biases = []
  weights = []
  for i in range(64):
    biases.append(random.randint(0,100)/100)
    temp1 = []
    for j in range(9216):
      temp1.append(random.randint(-500,500)/500)
    weights.append(temp1)
  data = {
      "biases":biases,
      "weights":weights
  }
  json.dump(data, write_file)

#Softmax
with open("softmax.json", "w") as write_file:
  biases = []
  weights = []
  for i in range(10):
    biases.append(random.randint(0,100)/100)
    temp1 = []
    for j in range(64):
      temp1.append(random.randint(-500,500)/500)
    weights.append(temp1)
  data = {
      "biases":biases,
      "weights":weights
  }
  json.dump(data, write_file)

# Array Allocation

In [3]:
def array1D(width,zeros=1):
    return [float(0.5-(i/width))*zeros for i in range(width)]

def array2D(height,width,zeros=1):
    return [array1D(width,zeros) for i in range(height)]

def array3D(depth,height,width,zeros=1):
    return [array2D(height,width,zeros) for i in range(depth)]

def array4D(tepth,depth,height,width,zeros=1):
    return [array3D(depth,height,width,zeros) for i in range(tepth)]

# Basic Utility Functions

In [4]:
def matrix_multiplication(mat1,mat2):
    result = array2D(len(mat1),len(mat2[0]))
    for i,x in enumerate(zip(mat1,mat2)):
        for j,y in enumerate(zip(x[0],x[1])):
            result[i][j] = y[0]*y[1]
    return result

def matrix_addition(mat1,mat2):
    result = []
    for i in range(len(mat1)):
        result.append(mat1[i]+mat2[i])
    return result

def matrix_addition2D(mat1,mat2):
    result = array2D(len(mat1),len(mat1[0]))
    for i in range(len(mat1)):
        for j in range(len(mat1[0])):
            result[i][j] = mat1[i][j]+mat2[i][j]
    return result

def matrix_subtraction(mat1,mat2):
    result = array2D(len(mat1),len(mat1[0]))
    for i in range(len(mat1)):
        for j in range(len(mat1[0])):
            result[i][j] = mat1[i][j]-mat2[i][j]
    return result

def matrix_subtraction3D(mat1,mat2):
    result = array3D(len(mat1),len(mat1[0]),len(mat1[0][0]))
    for i in range(len(mat1)):
        for j in range(len(mat1[0])):
            for k in range(len(mat1[0][0])):
                result[i][j][k] = mat1[i][j][k]-mat2[i][j][k]
    return result

def summation(mat1):
    result = 0
    for i in mat1:
        for j in i:
            result += j
    return result

def scaler_operation(image, operator, operand):
    result = array2D(len(image),len(image[0]))
    for i,x in enumerate(image):
        for j,y in enumerate(x):
            if operator == "div":
                result[i][j] = y/operand
            if operator == "mul":
                result[i][j] = y*operand
            if operator == "add":
                result[i][j] = y+operand
            if operator == "sub":
                result[i][j] = y-operand
            if operator == "pow":
                result[i][j] = y**operand
    return result

def scaler_operation3D(image, operator, operand):
    result = array3D(len(image),len(image[0]),len(image[0][0]))
    for k,z in enumerate(image):
        for i,x in enumerate(z):
            for j,y in enumerate(x):
                if operator == "div":
                    result[k][i][j] = y/operand
                if operator == "mul":
                    result[k][i][j] = y*operand
                if operator == "add":
                    result[k][i][j] = y+operand
                if operator == "sub":
                    result[k][i][j] = y-operand
                if operator == "pow":
                    result[k][i][j] = y**operand
    return result

def scaler_operation1D(image, operator, operand):
    result = array1D(len(image))
    for j,y in enumerate(image):
        if operator == "div":
            result[j] = y/operand
        if operator == "mul":
            result[j] = y*operand
        if operator == "add":
            result[j] = y+operand
        if operator == "sub":
            result[j] = y-operand
        if operator == "pow":
            result[j] = y**operand
    return result

def flatten(image):
    result = []
    for i in image:
        for j in i:
            for k in j:
                result.append(k)
    return result

def matrix_dot(A,B):
    result = array2D(len(A),len(B[0]))
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def matrix_exp(image):
    result = []
    for i in image:
        result.append(2.71828183**i)
    return result

def transpose(mat):
    result = array2D(len(mat[0]),len(mat))
    for i,x in enumerate(mat):
        for j,y in enumerate(x):
            result[j][i] = y
    return result

def reshape_matrix(mat,shape):
    h,w,d = shape
    result = array3D(h,w,d)
    indx = 0
    for i in range(h):
        for j in range(w):
            for k in range(d):
                result[i][j][k] = mat[indx]
                indx += 1
    return result

def image_normalization(im):
    temp = array2D(len(im),len(im[0]))
    for i,x in enumerate(im):
        for j,y in enumerate(x):
            temp[i][j] = (y/255)-0.5
    return temp

def amax(im):
    result = []
    for k in range(len(im[0][0])):
        temp = []
        for i in range(len(im)):
            for j in range(len(im[0])):
                temp.append(im[i][j][k])
        result.append(max(temp))
    return result

def argmax(arr):
    j,y = 0,arr[0]
    for i,x in enumerate(arr):
        if x > y:
            y = x
            j = i
    return j

def ln(x):
    n = 99999999.0
    return n * ((x ** (1/n)) - 1)

# Convolution Layer

In [6]:
class ConvLayer:
  def __init__(self,name,number_of_node,input_shape,filter_shape=(3,3)):
        self.number_of_node = number_of_node
        self.input_shape = input_shape
        self.filter_shape = filter_shape
        with open(name+".json", "r") as read_file:
          data = json.load(read_file)
        self.filters = data["weights"]
        self.biases = data["biases"]
  def forward(self,input_pattern):
    self.last_input_pattern = input_pattern
    output_pattern = array2D(self.input_shape[0]-2,self.input_shape[1]-2,zeros=0)
    for i in range(1,self.input_shape[0]-1):
      for j in range(1,self.input_shape[1]-1):
        temp = []
        for k in range(self.number_of_node):
          sum = 0
          for l in range(self.input_shape[2]):
            # Convolution and sumation
            sum += (input_pattern[i-1][j-1][l]*self.filters[k][l][0][0]+\
                    input_pattern[i-1][j][l]*self.filters[k][l][0][1]+\
                    input_pattern[i-1][j+1][l]*self.filters[k][l][0][2]+\
                    input_pattern[i][j-1][l]*self.filters[k][l][1][0]+\
                    input_pattern[i][j][l]*self.filters[k][l][1][1]+\
                    input_pattern[i][j+1][l]*self.filters[k][l][1][2]+\
                    input_pattern[i+1][j-1][l]*self.filters[k][l][2][0]+\
                    input_pattern[i+1][j][l]*self.filters[k][l][2][1]+\
                    input_pattern[i+1][j+1][l]*self.filters[k][l][2][2])/9.0
          sum += self.biases[k] #adding bias
          temp.append(self.activation(sum)) #activation
        output_pattern[i-1][j-1] = temp
    return output_pattern
  def activation(self,value):
    return 0 if value<0 else value

# Pooling Layer

In [7]:
class MaxPoolingLayer:
  def __init__(self, input_shape):
    self.input_shape = input_shape
    
  def forward(self, input_pattern):
    self.last_input_pattern = input_pattern
    output_pattern = array3D(self.input_shape[0]//2,self.input_shape[1]//2,self.input_shape[2])
    for i in range(self.input_shape[0]//2):
      for j in range(self.input_shape[1]//2):
        temp = []
        for k in range(self.input_shape[2]):
          temp.append(max(input_pattern[i*2][j*2][k],input_pattern[i*2][(j*2)+1][k],input_pattern[(i*2)+1][j*2][k],input_pattern[(i*2)+1][(j*2)+1][k]))
        output_pattern[i][j] = temp
    return output_pattern

# Dense Layer

In [14]:
class DenseLayer:
  def __init__(self,name,number_of_node,number_of_input):
    self.number_of_node = number_of_node
    self.number_of_input = number_of_input
    with open(name+".json", "r") as read_file:
          data = json.load(read_file)
    self.weights = data["weights"]
    self.biases = data["biases"]
  
  def forward(self,input_pattern):
    self.last_input_pattern = input_pattern
    output_pattern = []
    for i in range(self.number_of_node):
      sum = 0
      for j in range(self.number_of_input):
        sum += input_pattern[j]*self.weights[i][j]
      sum += self.biases[i]
      output_pattern.append(self.activation(sum))
    return output_pattern

  def activation(self,value):
    return 1/(1+2.71828183**(-1*value))

# SoftMax Layer

In [15]:
class SoftMax:
  def __init__(self,name,number_of_node,number_of_input):
    self.number_of_node = number_of_node
    self.number_of_input = number_of_input
    with open(name+".json", "r") as read_file:
          data = json.load(read_file)
    self.weights = data["weights"]
    self.biases = data["biases"]
  
  def activation(self,value):
    return 1/(1+2.71828183**(-1*value))
  
  def forward(self,input_pattern):
    self.last_input_pattern = input_pattern
    output_pattern = []
    for i in range(self.number_of_node):
      sum = 0
      for j in range(self.number_of_input):
        sum += input_pattern[j]*self.weights[i][j]
      sum += self.biases[i]
      output_pattern.append(self.activation(sum))
    self.last_output_patter =  output_pattern
    return output_pattern

  def backword(self,output,learn_rate):
    for i, gradient in enumerate(d_l_d_out):
      if(gradient == 0):
          continue
      
      #e^totals
      t_exp = matrix_exp(self.last_totals)
      #Sum of all e^totals
      S = sum(t_exp)

            #gradients of out[i] against totals
            #d_out_d_t = -t_exp[i] * t_exp/ (S**2)
      d_out_d_t = scaler_operation1D(scaler_operation1D(t_exp,"mul",-t_exp[i]),"div",S**2)
      d_out_d_t[i] = t_exp[i] * (S-t_exp[i]) /(S**2)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights
      
      #Gradients of loss against totals
      d_l_d_t =  scaler_operation1D(d_out_d_t,"mul",gradient)
      #Gradients of loss against weights/biases/input
            # print("dwt",transpose([d_t_d_w]))
      d_l_d_w = matrix_dot(transpose([d_t_d_w]),[d_l_d_t])
      d_l_d_b =  d_l_d_t
      d_l_d_inputs = transpose(matrix_dot(d_t_d_inputs , transpose([d_l_d_t])))[0]
      
      #update weights/biases
      self.weights =  matrix_subtraction(self.weights,scaler_operation(d_l_d_w,"mul",learn_rate))
      self.biases = matrix_subtraction([self.biases],[scaler_operation1D(d_l_d_b,"mul",learn_rate)])[0]
      return reshape_matrix(d_l_d_inputs,self.last_input_shape)

# Creating Model

In [16]:
conv1 = ConvLayer("conv1",32,(28,28,1))
conv2 = ConvLayer("conv2",64,(26,26,32))
maxpool2 = MaxPoolingLayer((24,24,64))
dense = DenseLayer("dense",64,9216)
softmax = SoftMax("softmax",10,64)

# Forwording

In [17]:
def forward(input_pattern,output_label):
  a = conv1.forward(input_pattern)
  b = conv2.forward(a)
  c = maxpool2.forward(b)
  d = flatten(c)
  e = dense.forward(d)
  output = softmax.forward(e)
  loss = -ln(output[output_label])
  acc = 1 if(argmax(output) == output_label) else 0
  return output, loss, acc

# Backpropagation

In [12]:
def backward(output, learn_rate):
  out = softmax.backward(output,learn_rate)


# Training

In [18]:
def training(x_train, y_train):
  for input_pattern,output_label in zip(x_train, y_train):
    output, loss, acc = forward(input_pattern,output_label)

    # gradient = array1D(10,zeros=0)
    # gradient[output_label] = -1/output[output_label]

    backward(output, 0.005)
    
    print(output_label,"--",output)

In [19]:
training(x_train, y_train)

5 -- [0.16488384664644276, 0.890913936740866, 0.9749019605163765, 0.9969061124650543, 0.22131555327341118, 0.1176845198190124, 0.9553846355499251, 0.49530564436532265, 0.9580817439973267, 0.01173443621820715]
0 -- [0.370606619456687, 0.7703353222823061, 0.9569926948203465, 0.9945342692931912, 0.17067673998030608, 0.1456920570740505, 0.929081493582041, 0.5993725407345029, 0.9769357796405516, 0.008741883567415067]
4 -- [0.22893281934158988, 0.8338010165265638, 0.9587255784926922, 0.9945592289101383, 0.1517942703222352, 0.14332962877784292, 0.9348970337684407, 0.5308223550156759, 0.9695784855033457, 0.009790993434108155]
1 -- [0.16698858756873575, 0.8693686996953922, 0.9669772596818411, 0.9962749982415846, 0.2089309896575143, 0.11557875263871495, 0.9531752727173062, 0.5443918651990393, 0.9698629544857295, 0.013728104910261786]
9 -- [0.18374712623407188, 0.8814210856715529, 0.971221892840399, 0.9966255783298812, 0.2181449807726725, 0.09639311660036559, 0.9543359937142553, 0.544358958019781

KeyboardInterrupt: ignored